In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#executing this code so that the client application Collecting Jobs API will be accessing this code executing on the server
!pip install flask
!wget  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/labs/module%201/Accessing%20Data%20Using%20APIs/jobs.json

In [ ]:
import flask
from flask import request, jsonify
import requests
import re

def get_data(key,value,current):
    results = list()
    pattern_dict = {
        'C'      : '(C)',
        'C++'    : '(C\+\+)',
        'Java'   :'(Java)',
        'C#'     : '(C\#)',
        'Python' :'(Python)',
        'Scala' : '(Scala)',
        'Oracle' : '(Oracle)',
        'SQL Server': '(SQL Server)',
        'MySQL Server' :'(MySQL Server)',
        'PostgreSQL':'(PostgreSQL)',
        'MongoDB'    : '(MongoDB)',
        'JavaScript'    : '(JavaScript)',
        'Los Angeles' :'(Los Angeles)',
        'New York':'(New York)',
        'San Francisco':'(San Francisco)',
        'Washington DC':'(Washington DC)',
        'Seattle':'(Seattle)',
        'Austin':'(Austin)',
        'Detroit':'(Detroit)',
        
        
        
        
        
    }
    for rec in current:
        print(rec[key])
        print(type(rec[key]))
        print(rec[key].find(value))
        #if rec[key].find(value) != -1:
        import re
        #reex_str = """(C)|(C\+\+)|(JavaScript)|(Java)|(C\#)|(Python)|(Scala)|(Oracle)|(SQL Server)|(MySQL Server)|(PostgreSQL)|(MongoDB)"""
        if re.search(pattern_dict[value],rec[key]) != None:
            results.append(rec)
    return results

app = flask.Flask(__name__)

import json
data = None
with open('jobs.json',encoding='utf-8') as f:
    # returns JSON object as
    # a dictionary
    data = json.load(f)
    
    

@app.route('/', methods=['GET'])
def home():
    
    return '''<h1>Welcome to flask JOB search API</p>'''


@app.route('/data/all', methods=['GET'])
def api_all():
    return jsonify(data)


@app.route('/data', methods=['GET'])
def api_id():
    # Check if keys such as Job Title,KeySkills, Role Category and others  are provided as part of the URL.
    #  Assign the keys to the corresponding variables..
    # If no key is provided, display an error in the browser.
    res = None
    for req in request.args:
        
        if req == 'Job Title':
            key = 'Job Title'
        elif req == 'Job Experience Required' :
            key='Job Experience Required'
        elif req == 'Key Skills' :
            key='Key Skills'
            
        elif req == 'Role Category' :
            key='Role Category'
        elif req == 'Location' :
            key='Location'
        
        elif req == 'Functional Area' :
            key='Functional Area'
        
        elif req == 'Industry' :
            key='Industry'
        elif req == 'Role' :
            key='Role'
        elif req=="id":
             key="id"
        else:
            pass
    
        value = request.args[key]
        if (res==None):
            res = get_data(key,value,data)
        else:
            res = get_data(key,value,res)

    # Use the jsonify function from Flask to convert our list of
    # Python dictionaries to the JSON format.
    return jsonify(res)

app.run()

In [ ]:
#Import required libraries
import pandas as pd
import json
import requests 

In [ ]:
#Write a function to get the number of jobs for the Python technology.
api_url="http://127.0.0.1:5000/data"
def get_number_of_jobs_T(technology):
    number_of_jobs = 0
    payload = {"Key Skills":technology}
    r = requests.get(api_url, params = payload)
    if r.ok:
        data = r.json()
        number_of_jobs+=len(data)
        
   
    return technology,number_of_jobs

In [ ]:
#Calling the function for Python and checking if it works.
get_number_of_jobs_T("Python")

In [ ]:
#Write a function to find number of jobs in US for a location of our choice.
def get_number_of_jobs_L(location):
    number_of_jobs = 0
    payload = {"Location":location}
    r = requests.get(api_url, params = payload)
    if r.ok:
        data = r.json()
        number_of_jobs=len(data)
    return location,number_of_jobs

In [ ]:
#Call the function for Los Angeles and check if it is working.

get_number_of_jobs_L("Los Angeles")

In [ ]:
#Store the results in an excel file
r = requests.get(api_url, params = 'Location')
print (r)


In [ ]:
locations = ['Los Angeles',
'New York',
'San Francisco',
'Washington DC',
'Seattle',
'Austin',
'Detroit']

In [ ]:
!pip install openpyxl
import openpyxl
from openpyxl import Workbook
wb = Workbook()
ws = wb.active
ws.append(['Job_Title', 'Location'])

In [ ]:
for location in locations:
    job_posting_count = get_number_of_jobs_L(location)
    print(job_posting_count)
    ws.append(['location','number_of_jobs'])
wb.save("job_postings.xlsx")   

In [ ]:
techs = ['C', 'C#', 'C++', 'Java', 'JavaScript', 'Python', 'Scala', 'Oracle', 'SQL Server', 'MySQL Server', 'PostgreSQL', 'MongoDB']
for tech in techs:
    job_counts = get_number_of_jobs_T (tech)
    ws.append(['technology','number_of_jobs'])
    print (job_counts)
wb.save("job_postings_tech.xlsx")
if ws.max_row > 3:
    print("Data was successfully written to the spreadsheet.")
else:
    print("No data was written to the spreadsheet.")